## Mood Recognition with HaarCascade

In [1]:
# Define the emotion dictionary
emotion_dict = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}  # Update with your actual emotions

def emotion_recog(frame, model, face_cascade_path='haarcascade_frontalface_default.xml'):
    # Prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # Load the Haar Cascade for face detection
    facecasc = cv2.CascadeClassifier(cv2.data.haarcascades + face_cascade_path)

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 255), 3)
        
        # Extract the region of interest (ROI) for emotion detection
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        
        # Make prediction on the ROI
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        
        # Annotate the frame with the predicted emotion
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return frame

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Path to your model file
# Load the model
model = load_model("C:/Users/kshit/Desktop/Emotion based song recc/emotion_model_1.h5")

In [6]:
import cv2
import numpy as np
import os

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break
    
    frame = cv2.flip(frame, 1)



    # Perform emotion recognition
    output_frame = emotion_recog(frame, model)

    # Display the resulting frame
    cv2.imshow('Emotion Recognition', output_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 26ms/step
